# Logs
In this notebook we will experiment with different ways of logging in Python.

A python program and its modules can be logged by many loggers. If no logger is specified and the logging module is called, the logging defaults to a `root` logger with `basicConfig`.

In general logs have the following components:
* [formatter](https://docs.python.org/3/library/logging.html#formatter-objects) - which formats the logs
  * format details can be found under [LogRecord attributes](https://docs.python.org/3/library/logging.html#logrecord-attributes)
* [handler](https://docs.python.org/3/library/logging.html#handler-objects) - which handles the logs - to either stream or a file
  * handlers can be formatted by attaching an appropriate formatter
* [filter](https://docs.python.org/3/library/logging.html#filter-objects) - a function / class that can be attached to handler(s) to suppress certain logs
* [level](https://docs.python.org/3/library/logging.html#logging-levels) - can be DEBUG, INFO, WARNING, CRITICAL and FATAL - as a standard
  * a level can be set to a handler
  * the levels of these handlers should be greater than the root's level for the logger to log

Loggers can be programmed within a python file or can use an external configuration file - which could be either in `dictionary` or `yaml` format.

Let us play with all these.


## Log to 2 files with a filter

Let us assume that we want our single program to log to console and two files - msg.log and all.log as follows:
* console - logs INFO and above, provided it does not contain the words 'no-console' in the message
* msg.log - logs INFO and above messages
* all.log - logs DEBUG and above messages

In [ ]:
import logging

logger = logging.getLogger(__name__)

Loggers are never instantiated directly. The module level function `logging.getLogger(name)` is always to be used.

By using `__name__` instead of a string, `main` for root or the name of the imported module is used for the log.

In [ ]:
logger.setLevel(logging.DEBUG) # it is important to set this to the lowest level required

Next we will prepare a common format for the log

In [ ]:
fmt = logging.Formatter("%(asctime)-15s %(name)-5s %(levelname)-8s %(message)s")

Next we will prepare a filter for the console log and instantiate it.

In [ ]:
class noConsoleFilter(logging.Filter):
    def filter(self, record):
        return not (record.levelname == 'INFO') & ('no-console' in record.msg)
f = noConsoleFilter()


And then we will set up the handlers, set its level and add filter for console only

In [ ]:
# Set up the console handler
ch = logging.StreamHandler() # create a console handler
ch.setLevel(logging.INFO) # Set its level
ch.setFormatter(fmt) # Set its format
ch.addFilter(f) # add the filter

# Set up the message file handler
fh_msg = logging.FileHandler(filename='./msg.log') # put the path
# Set msg.log's level, format and filter similar to console
fh_msg.setLevel(ch.level)
fh_msg.setFormatter(ch.formatter)

# Set up the all file handler
fh_all = logging.FileHandler(filename='./all.log') # put the path
fh_all.setFormatter(fmt) # set its format

...and lastly we will associate the handlers to the logger

In [ ]:
logger.addHandler(ch)  # to console
logger.addHandler(fh_msg) # to msg.log
logger.addHandler(fh_all) # to all.log

Let us check the code...

In [ ]:
if __name__ == "__main__":
    logger.warning("Warning message from main for Console, Msg and All logs")
    logger.info("Info message from main. Should log to Console, Msg and All logs")
    logger.info("no-console Info. Only for Msg and All logs")
    logger.debug("Debug. Should log only to all.log")

## Log to 2 files with filter and YAML config

Let us now try to configure the same through `log2file.yml`. We will configure a different log called `yml_logger` to test out.

In [ ]:
import logging
from setupLogging import setupLogging

setupLogging()
yml_logger = logging.getLogger(__name__)
yml_logger.info("Logger set :D")
